In [10]:
from snowpat import pysmet as smet
from snowpat import snowpackreader as spr
from snowpat import icsv
import pandas as pd

In [14]:
# Sample data
data = [
    [15.3, 60, 1013],  # T (Temperature), RH (Relative Humidity), P (Pressure)
    [14.8, 62, 1012],
    [16.1, 59, 1014]
]

# create a file object
file = icsv.iCSVFile()

# Convert to pandas DataFrame
data_pd = pd.DataFrame(data, columns=["T", "RH", "P"])
file.setData(data_pd)

#set minimal metadata
file.metadata.set_attribute["field_delimiter", ":"]
file.metadata.set_attribute["geometry", "POINT", ":"]
file.metadata.set_attribute["srid", "EPSG:1234", ":"]

file.info()
file.write("output.icsv")


File: None
METADATA:
Required:
field_delimiter : :
geometry : POINT(1 1)
srid : EPSG:1234
Recommended:

ACDD Attributes:
Unknown Attributes:
field_delimiter: :
geometry: POINT(1 1)
srid: EPSG:1234

Other Metadata:

Fields: ['T', 'RH', 'P']
Recommended Fields:

Other Fields:

Geometry: None
SRID: None
Location: X: None
Y: None
Z: None
EPSG: None

Data:
      0   1     2
0  15.3  60  1013
1  14.8  62  1012
2  16.1  59  1014


In [ ]:
#reading an iCSV
File = icsv.read(filename)
data_pandas = file.data
data_xarray = file.to_xarray()
# metadata and fields can be accessed with get_attribute
field_delimiter = file.metadata.get_attribute("field_delimiter")
fields = file.fields.get_attribute("fields")
# required keys will always be present, as a sanity check is done. Any other might return None if it is not available.
# To see what metadata is available, you can print the information:
file.info() # prints information about the whole file
print(file.metadata) # prints information on the metadata only
print(file.fields) # print information on the fields section

# changing metadata
file.metadata.set_attribute("field_delimiter", ":")

# and for writing to an output again (if no output filename is provided, the given filename is used with an out flag):
file.write(out_filename)

# It is possible to convert SMET files to iCSV:
from snowpat import pysmet as smet
smet_file = smet.read(smet_filename)
icsv_file = from_smet(smet_file)
